# Import properties from Wikidata

Used [Protege online](https://webprotege.stanford.edu/) to model the required classes and properties.

The resulting model was saved in OWL/XMLformat (`data/knowledge-graph.owl`).

Used [WebVOWL](http://vowl.visualdataweb.org/webvowl.html) to visualize the model.

![title](data/knowledge-graph.owl.svg)

1. Start a local wikibase, e.g. using [MaRDI4NFDI/portal-compose](https://github.com/MaRDI4NFDI/portal-compose)
2. Increase memory limit by setting `ini_set( 'memory_limit', '2G' );` in LocalSettings.d/LocalSettings.override.php

Possible ways to import items and properties from Wikidata (see https://www.mediawiki.org/wiki/Wikibase/Importing):
* ~Enable federated properties from Wikidata by setting `$wgWBRepoSettings['federatedPropertiesEnabled'] = true;` in Localsettings.d/LocalSettings.override.php~  (potentially creates entity namimg conflicts)
* ~Use native MediaWiki export/import to load the required pages fom Wikidata by importing the XML files into the local wikibase. These files contains item and property pages that have been exported from Wikidata.~ (potentially creates entity naming conflicts)
* Use WikibaseImport to import pages from Wikidata. ([older version](https://github.com/Wikidata/WikibaseImport/tree/c1233da6b7122c55c95d9d925b8f4162de8807e7) works, but I had to fork it, see below)
* ~Use a custom script to import the XML dumps from Wikidata into the local wiki without overwritting existing items and properties~ (last resort, as more work)

_Properties and items to import are listed in the file `data/Wikidata_import.csv`._

__Property pages that need to be imported from Wikidata__
* instance of (Property:P31)
* zbMATH author ID (Property:P1556)
* zbMATH work ID (Property:P894) 
* swMATH work ID (Property:P6830) 
* formatter URL (Property:P1630)
* DOI (Property:P356)
* describes a project that uses (Property:P4510)
* author (Property:P50)
* publication date (Property:P577)
* published in (Property:P1433) 
* publisher (Property:P123)
* title (Property:P1476)
* main subject (Property:P921) 
* language of work or name (Property:P407)
* volume (Property:P478)
* page(s) (Property:P304)
* issue (Property:P433)
* exact match (Property:P2888)

__Item pages that need to be imported from Wikidata__
* human (Q5)
* software (Q7397)
* scholarly article (Q13442814) 
* scientific publication (Q591041)
* publisher (Q2085381)
* index term (Q1128340) 
* language (Q34770)


# Importing using WikiImport
The properties and items to be imported have to be in a text file `data/Wikidata_import.csv`.
Since the import script will import not only the items listed in the file, but also all referenced properties and items, the list of imports might be quite large and take a while to complete. I [forked WikibaseImport](https://github.com/MaRDI4NFDI/WikibaseImport) to not import recursively, but only the items that are in the list.

## Try it out
To test the import manually, copy the list to the running Wikibase container:
```
docker cp Import\ from\ zbMath/data/Wikidata_import.csv mardi-wikibase:/tmp
```

Then run the import script. This script is a Wikibase extension, so runs from the Wikibase container:
```
docker exec -ti mardi-wikibase php extensions/WikibaseImport/maintenance/importEntities.php --file /tmp/Wikidata_import.csv --do-not-recurse
```


## Entity mapping
The properties and items imported from Wikidata are mapped to local ids. The mapping between original Wikidata ids and local ids is stored in the wiki's database in an extra table `wbs_entity_mapping`. For example:

```
MariaDB [my_wiki]> select * from wbs_entity_mapping limit 4;
+--------------+-----------------+
| wbs_local_id | wbs_original_id |
+--------------+-----------------+
| P76          | P10             |
| P119         | P1001           |
| P113         | P101            |
| P122         | P1011           |
+--------------+-----------------+
```

It would also be possible to add this mapping as a property to each entity page in the local Wikibase, but this should be backed by a specific use case.